In [86]:
import pandas as pd
import numpy as np

In [87]:
# Read computed fuel rates
df_trajectories = pd.read_csv(r'../../data/tidy/large/trajectories-mapped-powertrain-weight-grade-oct2021-sep2022.csv', delimiter=',', skiprows=0, low_memory=False)
df_trajectories.rename(columns={"VehiclWeight(lb)": "Vehicle_mass"}, inplace=True)
df_trajectories['Date']=pd.to_datetime(df_trajectories['Date'])
df_trajectories.speed = df_trajectories.speed *0.44704 # Convert from mph to m/s
df_trajectories = df_trajectories.fillna(0)

In [88]:
df_trajectories.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'ServiceDateTime', 'DateKey', 'CalendarDate', 'Year', 'Month', 'OpKey',
       'Operator', 'RtKey', 'RtKey2', 'Route', 'RouteName', 'TripKey', 'Trip',
       'StopKey', 'Stop', 'StopName', 'Vehicle', 'VehicleType', 'SeatedCap',
       'TotalCap', 'Lat', 'Lon', 'Boards', 'Alights', 'Onboard', 'Bike',
       'Daytype', 'Hour', 'Minute', 'Second', 'Date', 'Powertrain',
       'VehicleModel', 'Vehicle_mass', 'time_delta_in_seconds', 'dist',
       'speed', 'acc', 'elevation', 'grade'],
      dtype='object')

In [89]:
df_trajectories.drop(['Unnamed: 0.3'], axis=1, inplace=True)
df_trajectories.drop(['Unnamed: 0.2'], axis=1, inplace=True)
df_trajectories.drop(['Unnamed: 0.1'], axis=1, inplace=True)
df_trajectories.drop(['Unnamed: 0'], axis=1, inplace=True)
df_trajectories.head()

,ServiceDateTime,DateKey,CalendarDate,Year,Month,OpKey,Operator,RtKey,RtKey2,Route,...,Date,Powertrain,VehicleModel,Vehicle_mass,time_delta_in_seconds,dist,speed,acc,elevation,grade
0,2021-10-01 06:43:49,8578,2021-10-01 00:00:00.000,2021,10,1,VATCo,3386,1006,R44,...,2021-10-01,conventional,Gillig 30,25700,0,0.000000,0.000000,0.000000,45.0,0.000000
1,2021-10-01 06:51:53,8578,2021-10-01 00:00:00.000,2021,10,1,VATCo,3386,1006,R44,...,2021-10-01,conventional,Gillig 30,25700,484,0.594143,1.975578,0.002536,66.0,0.021959
2,2021-10-01 06:51:53,8578,2021-10-01 00:00:00.000,2021,10,1,VATCo,3387,3380,39E,...,2021-10-01,conventional,Gillig 30,25700,0,0.000000,0.000000,0.000000,66.0,0.000000
3,2021-10-01 06:52:53,8578,2021-10-01 00:00:00.000,2021,10,1,VATCo,3387,3380,39E,...,2021-10-01,conventional,Gillig 30,25700,60,0.313297,8.403368,0.087027,51.0,-0.029751
4,2021-10-01 06:54:38,8578,2021-10-01 00:00:00.000,2021,10,1,VATCo,3387,3380,39E,...,2021-10-01,conventional,Gillig 30,25700,105,0.421870,6.466031,-0.011465,51.0,0.000000


In [90]:
# BEB Validation
# read validation df
df_validation_BEB = pd.read_excel(r'../../data/tidy/BEB-validation.xlsx')
df_validation_BEB.rename(columns={"Transaction Date": "ServiceDateTime","Equipment ID":"Vehicle"}, inplace=True)
df_validation_BEB['ServiceDateTime'] = pd.to_datetime(df_validation_BEB['ServiceDateTime'])
df_validation_BEB['powertrain']='electric'
df_validation_BEB.rename(columns={"trip": "actual_energy"}, inplace=True)
df_validation_BEB.drop(['charge'], axis=1, inplace=True)
df_validation_BEB.loc[df_validation_BEB['actual_energy']!=0]
df_validation_BEB = df_validation_BEB[np.abs(df_validation_BEB['actual_energy']) > 1e-10]
df_validation_BEB

,Vehicle,actual_energy,ServiceDateTime,powertrain
0,3501,26.7,2022-06-01,electric
1,3501,167.1,2022-06-02,electric
3,3501,173.9,2022-06-06,electric
4,3501,74.3,2022-06-07,electric
5,3501,81.4,2022-06-08,electric
...,...,...,...,...
269,1321,141.5,2022-09-26,electric
270,1321,139.1,2022-09-27,electric
271,1321,194.8,2022-09-28,electric
272,1321,116.0,2022-09-29,electric


In [91]:
# CDB and HEB Validation
df_validation_CDB_HEB = pd.read_csv(r'../../data/tidy/fuel-tickets-clean-oct2021-sep2022.csv', delimiter=',', skiprows=0, low_memory=False)
df_validation_CDB_HEB=df_validation_CDB_HEB.loc[df_validation_CDB_HEB['Qty']>0]
df_validation_CDB_HEB.sort_values(by=['Equipment ID','Transaction Date'], inplace=True)
df_validation_CDB_HEB.drop(['Unnamed: 0'], axis=1, inplace=True)
df_validation_CDB_HEB.rename(columns={"Transaction Date": "ServiceDateTime","Equipment ID":"Vehicle"}, inplace=True)
df_validation_CDB_HEB.sort_values(by=['Vehicle','ServiceDateTime'], inplace=True)
df_validation_CDB_HEB['ServiceDateTime'] = pd.to_datetime(df_validation_CDB_HEB['ServiceDateTime'])
df_validation_CDB_HEB=df_validation_CDB_HEB.reset_index()

### Map powertrain in the validation dataset
df2 = pd.read_csv(r'../../data/tidy/vehicles-summary.csv', delimiter=',', skiprows=0, low_memory=False)
mydict = df2.groupby('Type')['Equipment ID'].agg(list).to_dict()
d = {val:key for key, lst in mydict.items() for val in lst}
df_validation_CDB_HEB['powertrain'] = df_validation_CDB_HEB['Vehicle'].map(d)
df_validation_CDB_HEB=df_validation_CDB_HEB.loc[df_validation_CDB_HEB['powertrain']!='electric']
df_validation_CDB_HEB.drop(['Meter1Reading'], axis=1, inplace=True)
df_validation_CDB_HEB.drop(['Fuel Orfluid Type'], axis=1, inplace=True)
df_validation_CDB_HEB.drop(['index'], axis=1, inplace=True)
df_validation_CDB_HEB.rename(columns={"Qty": "actual_energy"}, inplace=True)
df_validation_CDB_HEB.loc[df_validation_CDB_HEB['actual_energy']!=0]

df_validation_CDB_HEB

,Vehicle,actual_energy,ServiceDateTime,powertrain
389,1510,18.08,2022-01-10 15:30:00,conventional
390,1510,18.40,2022-01-11 22:01:00,conventional
391,1510,27.39,2022-01-12 19:21:00,conventional
392,1510,51.20,2022-01-13 18:04:00,conventional
393,1510,38.15,2022-01-14 19:36:00,conventional
...,...,...,...,...
27842,7902,76.20,2022-09-05 07:26:00,hybrid
27843,7902,34.00,2022-09-06 05:07:00,hybrid
27844,7902,50.00,2022-09-07 05:27:00,hybrid
27845,7902,23.30,2022-09-08 14:32:00,hybrid


In [92]:
### Integerating small actual_energy with the next observation

# Initialize an empty DataFrame to collect processed data
processed_data = pd.DataFrame(columns=df_validation_CDB_HEB.columns)

# Get unique vehicles
unique_vehicles = df_validation_CDB_HEB['Vehicle'].unique()

for vehicle in unique_vehicles:
    # Filter the DataFrame for the current vehicle
    temp_df = df_validation_CDB_HEB[df_validation_CDB_HEB['Vehicle'] == vehicle].reset_index(drop=True)
    
    # Iterate over the rows in the temporary DataFrame
    for i in range(len(temp_df) - 1):  # -1 because we will look at the next row
        if temp_df.at[i, 'actual_energy'] < 5:
            # Add the actual_energy to the next row
            temp_df.at[i + 1, 'actual_energy'] += temp_df.at[i, 'actual_energy']
        else:
            # If actual_energy >= 5, add the row to the processed DataFrame
            new_row = temp_df.iloc[i]
            processed_data = pd.concat([processed_data, pd.DataFrame([new_row])], ignore_index=True)
    
    # Always add the last row for each vehicle as there's no next row to modify
    last_row = temp_df.iloc[-1]
    processed_data = pd.concat([processed_data, pd.DataFrame([last_row])], ignore_index=True)

# The processed_data now contains all the updated entries
# Assign it back to df_validation_CDB_HEB if necessary
df_validation_CDB_HEB = processed_data.copy()

df_validation_CDB_HEB['Vehicle'] = df_validation_CDB_HEB['Vehicle'].astype('int64')

# Show the processed DataFrame
df_validation_CDB_HEB


,Vehicle,actual_energy,ServiceDateTime,powertrain
0,1510,18.08,2022-01-10 15:30:00,conventional
1,1510,18.40,2022-01-11 22:01:00,conventional
2,1510,27.39,2022-01-12 19:21:00,conventional
3,1510,51.20,2022-01-13 18:04:00,conventional
4,1510,38.15,2022-01-14 19:36:00,conventional
...,...,...,...,...
26701,7902,76.20,2022-09-05 07:26:00,hybrid
26702,7902,34.00,2022-09-06 05:07:00,hybrid
26703,7902,50.00,2022-09-07 05:27:00,hybrid
26704,7902,23.30,2022-09-08 14:32:00,hybrid


In [93]:
# Combine the two dataframes by adding them as rows
df_validation = pd.concat([df_validation_BEB, df_validation_CDB_HEB], ignore_index=True)
df_validation

,Vehicle,actual_energy,ServiceDateTime,powertrain
0,3501,26.7,2022-06-01 00:00:00,electric
1,3501,167.1,2022-06-02 00:00:00,electric
2,3501,173.9,2022-06-06 00:00:00,electric
3,3501,74.3,2022-06-07 00:00:00,electric
4,3501,81.4,2022-06-08 00:00:00,electric
...,...,...,...,...
26894,7902,76.2,2022-09-05 07:26:00,hybrid
26895,7902,34.0,2022-09-06 05:07:00,hybrid
26896,7902,50.0,2022-09-07 05:27:00,hybrid
26897,7902,23.3,2022-09-08 14:32:00,hybrid


In [94]:
df_trajectories.columns

Index(['ServiceDateTime', 'DateKey', 'CalendarDate', 'Year', 'Month', 'OpKey',
       'Operator', 'RtKey', 'RtKey2', 'Route', 'RouteName', 'TripKey', 'Trip',
       'StopKey', 'Stop', 'StopName', 'Vehicle', 'VehicleType', 'SeatedCap',
       'TotalCap', 'Lat', 'Lon', 'Boards', 'Alights', 'Onboard', 'Bike',
       'Daytype', 'Hour', 'Minute', 'Second', 'Date', 'Powertrain',
       'VehicleModel', 'Vehicle_mass', 'time_delta_in_seconds', 'dist',
       'speed', 'acc', 'elevation', 'grade'],
      dtype='object')

In [95]:
# Convert 'ServiceDateTime' to datetime if it's not already
df_validation['ServiceDateTime'] = pd.to_datetime(df_validation['ServiceDateTime'])
df_trajectories['ServiceDateTime'] = pd.to_datetime(df_trajectories['ServiceDateTime'])

# Make sure 'Vehicle' and 'ServiceDateTime' are sorted for efficient slicing later
df_trajectories = df_trajectories.sort_values(by=['Vehicle', 'ServiceDateTime'])

# Calculate averages and variances for each vehicle and time period in df_trajectories
grouped_trajectories = df_trajectories.groupby('Vehicle')
avg_trajectories = grouped_trajectories[['time_delta_in_seconds', 'dist', 'speed', 'acc', 'elevation', 'grade']].expanding().mean()
var_trajectories = grouped_trajectories[['time_delta_in_seconds', 'dist', 'speed', 'acc', 'elevation', 'grade']].expanding().var()

# Flatten the multi-index and reset index for merging
avg_trajectories.reset_index(inplace=True)
var_trajectories.reset_index(inplace=True)

# Rename columns for merging
avg_trajectories.columns = ['Vehicle', 'index'] + ['avg_' + col for col in avg_trajectories.columns[2:]]
var_trajectories.columns = ['Vehicle', 'index'] + ['var_' + col for col in var_trajectories.columns[2:]]

# Merge the average and variance data back into df_trajectories
df_trajectories = df_trajectories.reset_index().merge(avg_trajectories, on=['Vehicle', 'index'], how='left')
df_trajectories = df_trajectories.merge(var_trajectories, on=['Vehicle', 'index'], how='left')

# Drop 'index' as it's no longer needed
df_trajectories.drop('index', axis=1, inplace=True)

# Now, for each entry in df_validation, find the corresponding latest entry in df_trajectories
# This is done by merging on 'Vehicle' and ensuring 'ServiceDateTime' from trajectories is less than or equal to that in validation
merged_df = pd.merge_asof(df_validation.sort_values('ServiceDateTime'), df_trajectories.sort_values('ServiceDateTime'),
                          by='Vehicle', on='ServiceDateTime', direction='backward')

# Identify new average and variance columns, assuming they follow the 'avg_' and 'var_' naming convention
new_columns = [col for col in merged_df.columns if col.startswith('avg_') or col.startswith('var_')]

# Drop rows from merged_df where any of the new average or variance columns are NaN
final_df = merged_df.dropna(subset=new_columns)

# The result, final_df, contains your original validation data plus the latest averages and variances for each vehicle up to the time of validation.
final_df

,Vehicle,actual_energy,ServiceDateTime,powertrain,DateKey,CalendarDate,Year,Month,OpKey,Operator,...,avg_speed,avg_acc,avg_elevation,avg_grade,var_time_delta_in_seconds,var_dist,var_speed,var_acc,var_elevation,var_grade
0,7902,32.00,2021-10-01 05:26:00,hybrid,8578.0,2021-10-01 00:00:00.000,2021.0,10.0,1.0,VATCo,...,9.521032,0.002068,58.858268,0.000656,1.211804e+04,0.058443,57.046179,0.028738,370.852768,0.000340
2,7550,17.00,2021-10-01 14:03:00,conventional,8578.0,2021-10-01 00:00:00.000,2021.0,10.0,1.0,VATCo,...,3.738397,0.013123,63.857895,0.004825,2.135700e+04,0.137751,9.388089,0.003321,347.212503,0.000457
3,7660,14.90,2021-10-01 14:27:00,conventional,8578.0,2021-10-01 00:00:00.000,2021.0,10.0,1.0,VATCo,...,6.781219,0.011087,88.550000,-0.000384,2.942174e+04,0.267539,25.685361,0.004103,1635.539831,0.001251
4,1859,15.66,2021-10-01 15:17:00,conventional,8578.0,2021-10-01 00:00:00.000,2021.0,10.0,2.0,SATCo,...,8.543206,0.035734,43.214660,0.000715,1.703491e+04,0.087590,42.623519,0.048907,366.368498,0.001249
5,1557,20.69,2021-10-01 15:28:00,conventional,8578.0,2021-10-01 00:00:00.000,2021.0,10.0,2.0,SATCo,...,13.243309,0.054518,46.284404,0.000445,1.154965e+06,1.475382,60.687142,0.045891,399.623811,0.000307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26894,1717,40.73,2022-09-29 23:14:00,conventional,8941.0,2022-09-29 00:00:00.000,2022.0,9.0,2.0,SATCo,...,7.349412,0.033333,52.521363,-0.000278,1.216899e+05,0.153970,30.070940,0.028648,409.613819,0.001321
26895,1652,53.17,2022-09-29 23:20:00,conventional,8941.0,2022-09-29 00:00:00.000,2022.0,9.0,2.0,SATCo,...,7.953988,0.039740,50.911659,0.000685,2.255384e+05,0.167209,33.050204,0.036839,454.590981,0.001289
26896,1825,49.83,2022-09-29 23:34:00,conventional,8941.0,2022-09-29 00:00:00.000,2022.0,9.0,2.0,SATCo,...,7.757442,0.036161,47.733159,0.000640,1.214825e+05,0.115431,31.887693,0.045740,442.493150,0.001516
26897,1321,193.60,2022-09-30 00:00:00,electric,8941.0,2022-09-29 00:00:00.000,2022.0,9.0,2.0,SATCo,...,7.457768,0.041437,49.879975,0.000172,6.247126e+05,0.215123,30.965081,0.036741,471.519024,0.001156


In [96]:
final_df.columns

Index(['Vehicle', 'actual_energy', 'ServiceDateTime', 'powertrain', 'DateKey',
       'CalendarDate', 'Year', 'Month', 'OpKey', 'Operator', 'RtKey', 'RtKey2',
       'Route', 'RouteName', 'TripKey', 'Trip', 'StopKey', 'Stop', 'StopName',
       'VehicleType', 'SeatedCap', 'TotalCap', 'Lat', 'Lon', 'Boards',
       'Alights', 'Onboard', 'Bike', 'Daytype', 'Hour', 'Minute', 'Second',
       'Date', 'Powertrain', 'VehicleModel', 'Vehicle_mass',
       'time_delta_in_seconds', 'dist', 'speed', 'acc', 'elevation', 'grade',
       'avg_time_delta_in_seconds', 'avg_dist', 'avg_speed', 'avg_acc',
       'avg_elevation', 'avg_grade', 'var_time_delta_in_seconds', 'var_dist',
       'var_speed', 'var_acc', 'var_elevation', 'var_grade'],
      dtype='object')

In [97]:
#final_df = final_df.copy()
#final_df.drop(['ServiceDateTime'], axis=1, inplace=True)
#final_df

In [98]:
final_df.to_csv(r'../../data/tidy/multivariate-data-NN.csv', index=False)

In [100]:
lowest_energy_rows = df_validation_CDB_HEB.nsmallest(20, 'actual_energy')
lowest_energy_rows

,Vehicle,actual_energy,ServiceDateTime,powertrain
906,1559,0.04,2021-11-09 09:44:00,conventional
4350,1646,1.59,2022-09-14 16:35:00,conventional
4057,1642,2.03,2022-04-08 15:42:00,conventional
23040,7552,4.40,2022-09-03 14:12:00,conventional
22440,3308,4.89,2021-10-08 15:10:00,conventional
3601,1619,5.00,2021-10-26 16:22:00,conventional
7056,1670,5.00,2021-12-14 15:34:00,conventional
22854,7552,5.00,2022-01-19 14:26:00,conventional
25092,7812,5.00,2022-01-13 05:25:00,conventional
25993,7815,5.00,2022-05-21 13:44:00,conventional
